In [35]:
from bs4 import BeautifulSoup
from xmlr import xmliter
import tqdm
import re
import json
import pickle
from collections import defaultdict

In [39]:
xml = "./PostLinks.V1.3.xml"
# {'@Id': '11217966', '@CreationDate': '2013-02-18T03:03:17.867', 
# '@PostId': '114876', '@RelatedPostId': '21688', '@LinkTypeId': '3'}

# the link type: [1:related, 3:duplicate]
duplicate_map = defaultdict(list)
for attr in tqdm.tqdm(xmliter(xml, "row")):
    if attr["@LinkTypeId"] == "3":
        pid = attr["@PostId"]
        dpid = attr["@RelatedPostId"]
        duplicate_map[pid].append(dpid)
    elif attr["@LinkTypeId"] == "1":
        pass
    else:
        assert False

pickle.dump(duplicate_map, open("duplicate_map.pkl", 'wb'))

257022it [00:01, 184099.11it/s]


In [6]:
xml = "./Posts.V1.3.xml"
question_posts = {}
answer_posts = {}
skip_count = 0
for attr in tqdm.tqdm(xmliter(xml, "row")):
#question post
# {'@Id': '1', '@PostTypeId': '1', 
#'@CreationDate': '2010-07-20T19:09:27.200', 
#'@ViewCount': '8695', '@Score': '155', '@CommentCount': '0', 
#'@OwnerUserId': '10', 
#'@Title': 'What Does it Really Mean to Have Different Kinds of Infinities?', 
#'@Body': '<p>Can someone explain to me how there can be different kinds of infinities?</p>  <p>I was reading "<a href="http://en.wikipedia.org/wiki/The_Man_Who_Loved_Only_Numbers" rel="noreferrer">The man who loved only numbers</a>" by <a href="http://en.wikipedia.org/wiki/Paul_Hoffman_(science_writer)" rel="noreferrer">Paul Hoffman</a> and came across the concept of countable and uncountable infinities, but they\'re only words to me.</p>  <p>Any help would be appreciated.</p> ', 
#'@AnswerCount': '9', '@AcceptedAnswerId': '9', '@Tags': '<elementary-set-theory><intuition><infinity><faq>'}

#answer post
#<row Id="4" PostTypeId="2" 
#CreationDate="2010-07-20T19:14:10.603" Score="11" CommentCount="1" 
#OwnerUserId="31" 
#Body="&lt;p&gt;&lt;a href=&quot;http://www.bbc.co.uk/podcasts/series/moreorless&quot; rel=&quot;noreferrer&quot;&gt;More or Less&lt;/a&gt; is a BBC Radio 4 programme about maths and statistics in the news, and there is a free podcast. It's presented by &lt;a href=&quot;http://timharford.com/&quot; rel=&quot;noreferrer&quot;&gt;Tim Harford&lt;/a&gt;, the Undercover Economist from the &lt;a href=&quot;http://www.ft.com/home/uk&quot; rel=&quot;noreferrer&quot;&gt;Financial Times&lt;/a&gt;.&lt;/p&gt; " 
#ParentId="3"/>
    attrs = {}
    for k in attr:
        attrs[k.replace("@","")] = attr[k]
    ptype = attrs["PostTypeId"]
    if ptype == "1": # Question
        qid = attrs["Id"]
        question_posts[qid] = {
            "accept_aid":attrs["AcceptedAnswerId"] if "AcceptedAnswerId" in attrs else None,
            "answers": []
        }
    elif ptype == "2": # Answer
        aid = attrs["Id"]
        answer_posts[aid] = {
            "body": attrs["Body"],
            "pid": attrs["ParentId"],
            "score": attrs["Score"],
        }
    else:
        skip_count += 1
        
print(skip_count)

2466080it [01:11, 34687.97it/s]

0


In [7]:
pickle.dump(answer_posts, open("answer_posts.pkl", 'wb'))
pickle.dump(question_posts, open("question_posts.pkl", 'wb'))

In [40]:
# Check point here
duplicate_map = pickle.load(open('duplicate_map.pkl', 'rb'))
answer_posts = pickle.load(open("answer_posts.pkl", 'rb'))
question_posts = pickle.load(open("question_posts.pkl", 'rb'))

In [42]:
missing_qid = []
for aid in tqdm.tqdm(answer_posts):
    qid = answer_posts[aid]["pid"]
    if qid in question_posts:
        question_posts[qid]["answers"].append(aid)
    else:
        missing_qid.append(qid)
print(len(missing_qid))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1445495/1445495 [00:01<00:00, 1312077.58it/s]

8


In [44]:
for qid in tqdm.tqdm(question_posts):
    # for all questions without accepted answer, find its duplicate qustion accept answer if possible.
    current_accept = question_posts[qid]["accept_aid"]
    question_posts[qid]["all_accept_aid"] = [current_accept] if current_accept else []
    if qid in duplicate_map:
        for dqid in duplicate_map[qid]:
            if dqid in question_posts :
                dup_accept = question_posts[dqid]["accept_aid"]
                question_posts[qid]["all_accept_aid"].append(dup_accept)
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1020585/1020585 [00:02<00:00, 404577.21it/s]


In [45]:
for qid in tqdm.tqdm(question_posts):
    max_score = 0
    question_posts[qid]["best_answer"] = None
    for aid in question_posts[qid]["answers"]:
        score = float(answer_posts[aid]["score"])
        if score > max_score:
            question_posts[qid]["best_answer"] = aid
            max_score = score
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1020585/1020585 [00:02<00:00, 380870.76it/s]


In [46]:
pickle.dump(question_posts, open("question_posts.pkl", 'wb'))

In [50]:
aid2best_aid = {}
dummy_cnt = 0
for aid in tqdm.tqdm(answer_posts):
    qid = answer_posts[aid]["pid"]
    if qid in question_posts:
        all_accept_aid = question_posts[qid]["all_accept_aid"]
        if len(all_accept_aid) > 0:
            best_aid = all_accept_aid[0]
        else:
            best_aid = question_posts[qid]["best_answer"]
        aid2best_aid[aid] = best_aid if (best_aid and len(best_aid) > 0) else None
        if aid2best_aid[aid] is None or aid2best_aid[aid] not in answer_posts:
            # no "best" answer or its "best" answer has no record in train dataset.
            aid2best_aid[aid] = None
            dummy_cnt += 1
        elif question_posts[qid]["accept_aid"] is None and float(answer_posts[aid2best_aid[aid]]["score"]) < 2:
            # the "best" answer is from highly voted post, but the upvote is not very sufficient.
            aid2best_aid[aid] = None
            dummy_cnt += 1
    else:
        aid2best_aid[aid] = None
        dummy_cnt += 1
print(len(aid2best_aid), dummy_cnt / (len(aid2best_aid) + 1))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1445495/1445495 [00:02<00:00, 659323.14it/s]

1445495 0.17328100527431414


In [51]:
pickle.dump(aid2best_aid, open("aid2bestgt1_aid.pkl", 'wb'))

In [52]:
with open("aid2bestgt1_aid.pkl", 'rb') as fh:
    aid2best_aid = pickle.load(fh)
for i, key in enumerate(aid2best_aid.keys()):
    print(key, aid2best_aid[key])
    if i > 25: break

4 328809
7 7
9 9
13 9
14 9
15 281
16 7
17 17
24 17
26 17
30 49
31 49
35 9
38 1572
40 40
41 49
42 53
43 49
44 49
47 1572
48 40
49 49
52 9
53 53
54 40
55 1572
60 49


In [53]:
# Case when we have accepted answer just in the current thread
print(answer_posts['38'])
print(question_posts['29']["accept_aid"])
print(aid2best_aid['38'])

{'body': '<p>Lots of people like to use <a href="http://instacalc.com/">Instacalc</a> which lets you do unit conversions and store intermediate calculations in variables.</p> ', 'pid': '29', 'score': '7'}
1572
1572


In [54]:
# Case when there is not any accepted answer in the current thread and linked thread(s).
print(answer_posts['4'])
print(question_posts['3']["accept_aid"])
print(aid2best_aid['4'])

{'body': '<p><a href="http://www.bbc.co.uk/podcasts/series/moreorless" rel="noreferrer">More or Less</a> is a BBC Radio 4 programme about maths and statistics in the news, and there is a free podcast. It\'s presented by <a href="http://timharford.com/" rel="noreferrer">Tim Harford</a>, the Undercover Economist from the <a href="http://www.ft.com/home/uk" rel="noreferrer">Financial Times</a>.</p> ', 'pid': '3', 'score': '11'}
None
328809


In [55]:
# Case when the "best" answer is from highly voted post, but the upvote is not very sufficient.
print(answer_posts['889'])
print(question_posts['874']["accept_aid"])
print(aid2best_aid['889'])

{'body': '<p>If it is just <span class="math-container" id="9014">45</span> degrees, then the answer is not very difficult. Center a coordinate system at the bottom left hand corner of the rectangle. Hence the coordinates of the (???) point are <span class="math-container" id="9015">(q,0)</span> for some <span class="math-container" id="9016">q&lt;w</span>.</p>  <p>Note that the because theta is <span class="math-container" id="9017">45</span> degrees, <span class="math-container" id="9018">y=w-q</span> (Isosceles right triangle). Hence <span class="math-container" id="9019">q=w-y</span>, and our point is simply <span class="math-container" id="9020">(w-y, 0)\\dots</span></p> ', 'pid': '874', 'score': '1'}
None
None
